In [1]:
import sys
sys.path.append('/source/main/')

In [2]:
import logging
logging.basicConfig(level=logging.INFO)

import numpy as np
import torch

from data_for_train import dataset as my_dataset
from model_def.seq2seq_attn import Seq2SeqAttn


In [3]:
my_dataset.bootstrap()

INFO:root:Indexing vocabs successfully. Total vocabs: 25390
INFO:root:Indexing vocabs successfully. Total vocabs: 50437
INFO:root:Vocab for source from file /source/main/vocab/output/src.pkl contains 25390 tokens
INFO:root:Vocab for source from file /source/main/vocab/output/tgt.pkl contains 50437 tokens


In [14]:
model = Seq2SeqAttn(src_vocab_size=len(my_dataset.voc_src.index2word),
                    tgt_vocab_size=len(my_dataset.voc_tgt.index2word),
                    start_idx=2,
                    end_idx=3
                    )


In [15]:
model.start_idx

tensor([[2]])

In [16]:
model.start_idx = torch.Tensor([[10]])

In [17]:
model.start_idx

tensor([[10.]])

In [18]:
device = torch.device('cpu')
PRE_TRAINED_MODEL = '/source/main/train/output/saved_models//Seq2SeqAttn/2019-05-30T06:27:24/210000.pt'

checkpoint = torch.load(PRE_TRAINED_MODEL, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

logging.info('Load pre-trained model from %s successfully', PRE_TRAINED_MODEL)

INFO:root:Load pre-trained model from /source/main/train/output/saved_models//Seq2SeqAttn/2019-05-30T06:27:24/210000.pt successfully


In [19]:
model.start_idx

tensor([[2.]])

In [12]:
def predict_batch(input_docs):
    input_idx = my_dataset.voc_src.docs2idx(input_docs, equal_length=100)
    input_tensor = torch.from_numpy(np.array(input_idx))
    output_tensor = model(input_tensor)
    output_idx = output_tensor.cpu().numpy()    
    output_docs = my_dataset.voc_tgt.idx2docs(output_idx)
    return output_docs

In [15]:
predict_batch(['hom nay toi di cho'])

['hôm nay tôi đi chợ']

In [12]:
model

Seq2SeqAttn(
  (encoder): Encoder(
    (embedding): Embedding(25390, 256)
    (lstm): LSTM(256, 512, num_layers=3, dropout=0.3, bidirectional=True)
    (dropout): Dropout(p=0.3)
  )
  (flatten_hidden_lstm): FlattenHiddenLSTM()
  (core_decoder): AttnRawDecoder(
    (embedding): Embedding(50437, 256)
    (lstm): LSTM(256, 512, num_layers=3, dropout=0.3)
    (attention): Attention(
      (scoring): Linear(in_features=512, out_features=1024, bias=True)
      (softmax): Softmax()
    )
    (output_mapping): Linear(in_features=1536, out_features=50437, bias=True)
    (dropout): Dropout(p=0.3)
  )
  (greedy_infer): DecoderGreedyInfer(
    (core_decoder): AttnRawDecoder(
      (embedding): Embedding(50437, 256)
      (lstm): LSTM(256, 512, num_layers=3, dropout=0.3)
      (attention): Attention(
        (scoring): Linear(in_features=512, out_features=1024, bias=True)
        (softmax): Softmax()
      )
      (output_mapping): Linear(in_features=1536, out_features=50437, bias=True)
      (drop